In [86]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [33]:
df = pd.read_csv("~/Documentos/ktm/data/dummy/data.csv")
df

,user,item,skill,correct,wins,fails
0,1,1,1,1,0,0
1,1,2,2,0,0,0
2,2,1,1,0,0,0
3,2,1,1,1,0,1
4,2,2,2,0,0,0


Onehot encoding wins and fails per skill

In [25]:
skills = df['skill'].unique()
skills_array = skills.reshape(-1, 1)
skills

array([1, 2])

In [29]:
np.where(skills == 2)[0][0]

1

In [28]:
enc = OneHotEncoder(sparse=False)
skills_onehot = enc.fit_transform(skills_array)
skills_onehot

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0.],
       [0., 1.]])

In [39]:
skills_onehot[0]

array([1., 0.])

In [76]:
def create_onehot(row):
    idx = np.where(skills == row['skill'])[0][0]
    wins = row['wins']*skills_onehot[idx]
    fails = row['fails']*skills_onehot[idx]
    return np.concatenate((skills_onehot[idx], wins, fails))

onehot_array = df.apply(create_onehot, axis=1)
cols = ["skills_%d"%skill for skill in skills]
cols += ["wins_%d"%skill for skill in skills]
cols += ["fails_%d"%skill for skill in skills]

['skills_1', 'skills_2', 'wins_1', 'wins_2', 'fails_1', 'fails_2']

In [81]:
onehot_df = pd.DataFrame(onehot_array.tolist(), columns=cols)
df = pd.concat((df,onehot_df), axis=1)
df

,user,item,skill,correct,wins,fails,skills_1,skills_2,wins_1,wins_2,fails_1,fails_2
0,1,1,1,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,1,1,1,0,1,1.0,0.0,0.0,0.0,1.0,0.0
4,2,2,2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [82]:
X = df[cols]
y = df['correct']

In [89]:
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm

In [85]:
model = LogisticRegression(fit_intercept = False)
mdl = model.fit(X, y)
model.coef_

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[ 0.23760452, -0.67483225,  0.        ,  0.        ,  0.35584349,
         0.        ]])

In [90]:
logit = sm.Logit(y, X)
logit.fit().params

         Current function value: 0.277259
         Iterations: 35


LinAlgError: Singular matrix